## Chapter 6 Exercise 07
Train and fine-tune a Decision Tree for the moons dataset.

a. Generate a moons dataset using make_moons(n_samples=10000, noise=0.4).

b. Split it into a training set and a test set using train_test_split().

c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [1]:
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Get data set
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4)

In [3]:
# split train test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

In [4]:
# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
params = [{
#     'criterion' : ['gini', 'entropy'],
    'max_leaf_nodes' : [None, 3, 10, 30, 100],
    'min_samples_leaf' : [3, 10, 30, 100],
#     'max_depth' : [3, 10, 30, 100],
}]
dt_clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_clf, params, cv=3, scoring='accuracy', n_jobs=-1, return_train_score=True)
grid_search.fit(X_train, y_train)

# print result
cvres = grid_search.cv_results_
for mean_score, params in sorted(zip(cvres['mean_test_score'], cvres['params']), key=lambda x: x[0], reverse=True):
    print(mean_score, params)

0.8596666666666667 {'max_leaf_nodes': None, 'min_samples_leaf': 30}
0.8596666666666667 {'max_leaf_nodes': 100, 'min_samples_leaf': 30}
0.8591666666666666 {'max_leaf_nodes': 30, 'min_samples_leaf': 3}
0.8583333333333333 {'max_leaf_nodes': 30, 'min_samples_leaf': 10}
0.8575 {'max_leaf_nodes': 30, 'min_samples_leaf': 30}
0.8563333333333333 {'max_leaf_nodes': None, 'min_samples_leaf': 100}
0.8563333333333333 {'max_leaf_nodes': 30, 'min_samples_leaf': 100}
0.8563333333333333 {'max_leaf_nodes': 100, 'min_samples_leaf': 100}
0.8535 {'max_leaf_nodes': 10, 'min_samples_leaf': 100}
0.8525 {'max_leaf_nodes': 10, 'min_samples_leaf': 3}
0.8525 {'max_leaf_nodes': 10, 'min_samples_leaf': 10}
0.8525 {'max_leaf_nodes': 10, 'min_samples_leaf': 30}
0.8456666666666667 {'max_leaf_nodes': None, 'min_samples_leaf': 10}
0.8455 {'max_leaf_nodes': 100, 'min_samples_leaf': 10}
0.8415 {'max_leaf_nodes': 100, 'min_samples_leaf': 3}
0.8253333333333334 {'max_leaf_nodes': 3, 'min_samples_leaf': 3}
0.8253333333333334 

In [5]:
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(max_leaf_nodes=100, min_samples_leaf=30, random_state=42)
dt_clf.fit(X_train, y_train)
y_pred = dt_clf.predict(X_test)
print('accuracy:', accuracy_score(y_pred, y_test))

accuracy: 0.854


## 8. Grow a forest.
a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit- Learn’s ShuffleSplit class for this.

b. Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This gives you majority-vote predictions over the test set.

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [6]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [31]:
from sklearn.base import clone

accumulated_accuracy = 0
forest = []
for mini_train_set in mini_sets:
    X_mini_train = mini_train_set[0]
    y_mini_train = mini_train_set[1]
    # create a tree
    tree = DecisionTreeClassifier(max_leaf_nodes=100, min_samples_leaf=30, random_state=42)
    tree.fit(X_mini_train, y_mini_train)
    forest.append(tree)
    
    y_mini_pred = tree.predict(X_test)
    accumulated_accuracy += accuracy_score(y_mini_pred, y_test)
    
print("average accuracy:", accumulated_accuracy / len(forest))

average accuracy: 0.7641089999999994


In [39]:
from scipy.stats import mode
def forest_predict(X):
    predicts = []
    for tree in forest:
        predicts.append(tree.predict([X]))
    return mode(predicts)[0][0][0]

In [41]:
y_pred = []
for test_set in X_test:
    y_pred.append(forest_predict(test_set))
print('forest predict accuracy:', accuracy_score(y_pred, y_test))

forest predict accuracy: 0.7745


In [42]:
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

NameError: name 'grid_search_cv' is not defined